In [ ]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np
import jupyter_helper_functions
import string
import time

load_dotenv()

True

# 2. Load and Explore EudraCT Data
Load the EudraCT dataset and perform exploratory data analysis to understand its structure and content.

In [ ]:
corrupt_dataset = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/parker-repair/eudract_repair.csv"
)
gold_standard = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/eudract_cleaned_gold_first1000.csv"
)

partial_keys = jupyter_helper_functions.load_text_file(
    "../datasets/parker_datasets/eudract/eudract.partialkey"
)
rules = jupyter_helper_functions.load_text_file(
    "../datasets/parker_datasets/eudract/eudract.rules"
)

# print(partial_keys)
# print(rules)
# print(corrupt_dataset.head(2))
# print(gold_standard.head(2))
# print(type(gold_standard.get("eudract_number").iloc[0]))
# print(type(corrupt_dataset.get("eudract_number").iloc[0]))

eudract_number ->
    single_blind,
    double_blind,
    open,
    controlled,
    placebo,
    active_comparator,
    randomised,
    crossover,
    parallel_group,
    arms

-- Attributes
@open:STRING
@single_blind:STRING
@double_blind:STRING
@randomised:STRING
@controlled:STRING
@placebo:STRING
@active_comparator:STRING
@crossover:STRING
@parallel_group:STRING
@arms:STRING

-- Overview of attributes
open notin {'Yes', 'No'}
single_blind notin {'Yes', 'No'}
double_blind notin {'Yes', 'No'}
randomised notin {'Yes', 'No'}
controlled notin {'Yes', 'No'}
placebo notin {'Yes', 'No'}
active_comparator notin {'Yes', 'No'}
crossover notin {'Yes', 'No'}
parallel_group notin {'Yes', 'No'}
arms notin {'0', '1', '2+'}

-- eudract rules for masking
open == 'Yes' & single_blind == 'Yes'
open == 'Yes' & double_blind == 'Yes'
single_blind == 'Yes' & double_blind == 'Yes'
open == 'No' & single_blind == 'No' & double_blind == 'No'

-- eudract rules for control
controlled == 'No' & placebo == 'Yes'
co

# 3. Clean and Merge Data with LLM
Use the LLM pipeline to clean and merge the corrupted dataset using the provided rules and evaluate the results.

In [ ]:
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models
import string
import json
import jupyter_helper_functions
string.punctuation = string.punctuation.replace("'", "")  # Remove single quotes from punctuation

# Use a primary key for merging
primary_key = "eudract_number"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
rows_of_context = 50

extra = "parker_postprocessing_option_2"
file_name = jupyter_helper_functions.sanitize_filename(f"{model.value}_{rows_of_context}_rows_context_{extra}")   

rpm = 30
additional_prompt = f"""
Here are rows of the dataset to provide context for the cleaning process:
{corrupt_dataset.sample(rows_of_context).to_string(index=False)}
"""

# Merge/clean with LLM
merged_df, time_taken = jupyter_helper_functions.merge_with_llm_timed(
    dataset = corrupt_dataset,
    primary_key = primary_key,
    model = model,
    rpm = rpm,
    additional_prompt = additional_prompt
    )


Merging groups with LLM:   0%|          | 0/524 [00:00<?, ?it/s]

Merging groups with LLM:  28%|██▊       | 149/524 [04:58<12:31,  2.00s/it]


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.0-flash-lite', 'location': 'global'}, 'quotaValue': '30'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '51s'}]}}

In [ ]:
# Save merged dataset
jupyter_helper_functions.save_dataframe_csv(merged_df, f"../analysis/repairs/eudract/merged_dataset_{file_name}.csv")

original_corrupted_datasets = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/eudract_corrupted_first1000.csv"
)
# Evaluate results
jupyter_helper_functions.standardize_and_evaluate(
    gold_standard=gold_standard,
    merged_df=merged_df,
    corrupt_dataset=original_corrupted_datasets,
    primary_key=primary_key,
    time_delta=time_taken,
    results_dir=f"../analysis/results/eudract/",
    file_name=file_name,
)

NameError: name 'merged_df' is not defined